<a href="https://colab.research.google.com/github/Oriolrt/MVC_M2_PGM/blob/main/3_Earthquake_Example_Sampling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example Using the Earthquake network
In this example I will try to create the Alarm Bayesian Network using pgmpy and do some simple queries on the network. This network is mentioned in Bayesian Artificial Intelligence - Section 2.5.1 (https://bayesian-intelligence.com/publications/bai/book/BAI_Chapter2.pdf).
The actual model in this notebook and the CPD tables are the ones used in C2 of the Master of Computer Vision given by the Universitat Autònoma de Barcelona (UAB)  

In [ ]:
!pip install pgmpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 14.7 MB/s eta 0:00:00


In [ ]:
# Importing Library
from pgmpy.models import BayesianNetwork
from pgmpy.inference import VariableElimination
from pgmpy.metrics.bn_inference import BayesianModelProbability
from pgmpy.inference import BeliefPropagation


In [ ]:
# Defining network structure

alarm_model = BayesianNetwork(
    [
        ("Burglary", "Alarm"),
        ("Earthquake", "Alarm"),
        ("Alarm", "NeighborCalls"),
    ]
)

# Defining the parameters using CPT
from pgmpy.factors.discrete import TabularCPD

cpd_burglary = TabularCPD(
    variable="Burglary", variable_card=2, values=[[0.7], [0.3]]
)
cpd_earthquake = TabularCPD(
    variable="Earthquake", variable_card=2, values=[[0.9], [0.1]]
)
cpd_alarm = TabularCPD(
    variable="Alarm",
    variable_card=2,
    values=[[0.99, 0.1, 0.3, 0.01], [0.01, 0.9, 0.7, 0.99]],
    evidence=["Burglary", "Earthquake"],
    evidence_card=[2, 2],
)
cpd_neighborcalls = TabularCPD(
    variable="NeighborCalls",
    variable_card=2,
    values=[[0.9, 0.2], [0.1, 0.8]],
    evidence=["Alarm"],
    evidence_card=[2],
    state_names={"NeighborCalls":["no","yes"],"Alarm":[False,True]},
)

# Associating the parameters with the model structure
alarm_model.add_cpds(
    cpd_burglary, cpd_earthquake, cpd_alarm, cpd_neighborcalls
)

In [ ]:
print(cpd_earthquake)
print(cpd_burglary)
print(cpd_alarm)
print(cpd_neighborcalls)



+---------------+-----+
| Earthquake(0) | 0.9 |
+---------------+-----+
| Earthquake(1) | 0.1 |
+---------------+-----+
+-------------+-----+
| Burglary(0) | 0.7 |
+-------------+-----+
| Burglary(1) | 0.3 |
+-------------+-----+
+------------+---------------+---------------+---------------+---------------+
| Burglary   | Burglary(0)   | Burglary(0)   | Burglary(1)   | Burglary(1)   |
+------------+---------------+---------------+---------------+---------------+
| Earthquake | Earthquake(0) | Earthquake(1) | Earthquake(0) | Earthquake(1) |
+------------+---------------+---------------+---------------+---------------+
| Alarm(0)   | 0.99          | 0.1           | 0.3           | 0.01          |
+------------+---------------+---------------+---------------+---------------+
| Alarm(1)   | 0.01          | 0.9           | 0.7           | 0.99          |
+------------+---------------+---------------+---------------+---------------+
+--------------------+--------------+-------------+
| Alarm

# Sampling models

## Gibbs Sampling

In [ ]:
from pgmpy.sampling import GibbsSampling
gibbs_chain = GibbsSampling(alarm_model)

gibbs_chain.sample(size=10)

  0%|          | 0/9 [00:00<?, ?it/s]

,Burglary,Alarm,Earthquake,NeighborCalls
0,0,0,1,0
1,0,1,1,1
2,0,1,0,1
3,1,1,1,1
4,0,1,1,1
5,0,1,1,1
6,0,1,1,1
7,1,1,0,0
8,1,0,0,0
9,0,0,0,0


## Approximate Inference

Belief propagation results

In [ ]:
from pgmpy.inference import BeliefPropagation
alarm_infer = BeliefPropagation(alarm_model)

# Computing the probability of a Burglary given NeighborCalls="yes".
q=alarm_infer.query(variables=["Burglary"], evidence={"NeighborCalls": "yes"})
print(q)

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.3929 |
+-------------+-----------------+
| Burglary(1) |          0.6071 |
+-------------+-----------------+


In [ ]:
from pgmpy.inference.ApproxInference import ApproxInference
infer = ApproxInference(alarm_model)

q = infer.query(variables=["Burglary"],evidence={'NeighborCalls':'yes'})
print(q)

  0%|          | 0/10000 [00:00<?, ?it/s]

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(1) |          0.6093 |
+-------------+-----------------+
| Burglary(0) |          0.3907 |
+-------------+-----------------+


In [ ]:

q = infer.query(variables=["Burglary"])
print(q)

q_bp = alarm_infer.query(variables=["Burglary"])
print(q_bp)

  0%|          | 0/4 [00:00<?, ?it/s]

+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7120 |
+-------------+-----------------+
| Burglary(1) |          0.2880 |
+-------------+-----------------+
+-------------+-----------------+
| Burglary    |   phi(Burglary) |
+=============+=================+
| Burglary(0) |          0.7000 |
+-------------+-----------------+
| Burglary(1) |          0.3000 |
+-------------+-----------------+
